# Install Necessary Files

In [ ]:
!pip install langchain_groq
!pip install langchain_community
!pip install chromadb
!pip install fastapi
!pip install nest_asyncio
!pip install uvicorn
!pip install pyngrok

# Import Libraries

In [ ]:
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio
import uuid
import pandas as pd
import chromadb
from fastapi import FastAPI, Request
from fastapi.responses import HTMLResponse
from pydantic import BaseModel
from pyngrok import ngrok
from langchain_groq import ChatGroq
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import PromptTemplate

# Set up Fast API

In [ ]:
nest_asyncio.apply()

app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=["http://localhost:5173"],  # frontend origin
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Set up Groq with llama

In [ ]:
llm = ChatGroq(
    temperature=0,
    groq_api_key='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',  # Replace with your key
    model_name="llama3-70b-8192"
)

# Read CSV File

In [ ]:
df = pd.read_csv('/kaggle/input/dataset01/350links.csv')

# Set up ChromaDB

In [ ]:
client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

for _, row in df.iterrows():
    collection.add(documents=row["Keywords"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

# Implementation of Langchain Logic

In [ ]:
def get_answer(question: str) -> str:
    try:
        # Rephrase the question if needed
        prompt_user_1 = PromptTemplate.from_template(
            """
            ### Question:
            {question}
            ### INSTRUCTION:
            Rephrase it if needed for your convinience otherwise not needed, no preamble.
            ### QUESTION_ANSWER (NO PREAMBLE):
            """
        )

        # Create a chain to rephrase the question
        chain_question = prompt_user_1 | llm
        rephrased = chain_question.invoke({"question": question}).content

        # Search for relevant links in the collection
        links = collection.query(query_texts=[rephrased], n_results=1).get("metadatas", [])
        if not links or not links[0]:
            return "No relevant data found."

        link = links[0][0]['links']
        loader = WebBaseLoader(str(link))
        raw_content = loader.load().pop().page_content
        page_data = " ".join(raw_content.split())

        # Create a prompt to answer the question using the page data
        prompt_user = PromptTemplate.from_template(
            """
            ### Question:
            {question}
            ### INSTRUCTION:
            Answer only using {info}. No preamble.
            ### QUESTION_ANSWER (NO PREAMBLE):
            """
        )

        # Create a chain to answer the question
        chain_answer = prompt_user | llm
        res = chain_answer.invoke({"question": rephrased, "info": page_data.strip()})
        return res.content.strip()

    except Exception as e:
        return f"Error: {str(e)}"

# Define FastAPI Endpoints

In [ ]:
class Question(BaseModel):
    question: str


# Ask Routing
@app.post("/ask")
async def ask_question(q: Question):
    answer = get_answer(q.question)
    return {"answer": answer}


# Make HTML Page to Render the Question and Answer
@app.get("/", response_class=HTMLResponse)
async def serve_html():
    return HTMLResponse(content="""
    <html>
    <body>
        <h2>Ask a Question</h2>
        <input type="text" id="q" size="50">
        <button onclick="ask()">Submit</button>
        <p><b>Answer:</b> <span id="a"></span></p>
        <script>
        async function ask() {
            let q = document.getElementById("q").value;
            let res = await fetch("/ask", {
                method: "POST",
                headers: {
                    "Content-Type": "application/json",
                    "ngrok-skip-browser-warning": "true"
                },
                body: JSON.stringify({ question: q })
            });
            let json = await res.json();
            document.getElementById("a").innerText = json.answer;
        }
        </script>
    </body>
    </html>
    """)

# Start FastAPI with Public URL

In [ ]:
# Start FastAPI with public URL
from uvicorn import Config, Server
!ngrok config add-authtoken '2xtQ4VTsnytY20U2JGk9ck2dqzO_3dTrMvdoZUu5jUJVJucJn'
public_url = ngrok.connect(8000)
print("Your chatbot is live at:", public_url)


# Start FastAPI Server
config = Config(app=app, port=8000, log_level="info")
server = Server(config=config)
await server.serve()